In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

from wordcloud import WordCloud
from konlpy.tag import Okt
from hanspell import spell_checker

In [2]:
from soynlp.normalizer import *
import urllib.request
from soyspacing.countbase import CountSpace,RuleDict
import soyspacing

## 데이터 불러오기

In [3]:
data=pd.read_csv('./data/Lyrics_trans.csv',encoding='utf-8')
data

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0
1,라라랄라 라라랄라 워어 워어 너가 맛있었다고 했던 살구빛 샐러드 맛있어서 또 먹고 ...,0
2,자꾸 마주치는 네 눈빛에 아기 아기왠지 어색해 대체 왜 이래고개를 숙여봐도너의 발끝...,0
3,어깨동무까진 괜찮아그치만 손 잡는 건 조금 위험해잘 자란 메세진 괜찮아그래도 자냐는...,0
4,햇살이 가득그댈 향한 가벼운 발걸음콧노래 부르며쇼윈도에 비친내 모습을 한번 점검해시...,0
...,...,...
741,반쯤 미쳐가는 나망가져 나의 삶넌 내게서 나를 지워가고또 나를 부셔버려라추락한 나를...,3
742,무궁화 꽃이 피었습니다우리 빼고 모두 다 멈춰지담이 꽃이 피었습니다현아 빼고 모두 ...,3
743,아예 아예 아예 아예 아예 아예 아예 아예 아예 아예 왜 이렇게 감을 못 잡는거야 ...,3
744,난 나쁜 기집애 나난 나쁜 기집애 난 나쁜 기집애 내 모든 나쁜 여자가있는 곳난 나...,3


normalizer

In [4]:
lyrics=[]
for lyric in data['Lyrics']:
    lyrics.append(repeat_normalize(lyric,num_repeats=1))
len(lyrics)

746

띄어쓰기1차 - soynlp사용

In [5]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")
model=CountSpace()
model.train('./2016-10-20.txt')

all tags length = 1909391 --> 1671744, (num_doc = 30090))

In [6]:
result=[]
for lyric in lyrics:
    sub,tags=model.correct(lyric,verbose=False,force_abs_threshold=0.3,nonspace_threshold=-0.3,space_threshold=0.3,min_count=10)
    result.append(sub)
len(result)

746

띄어쓰기2차 - hanspell사용

In [7]:
lyrics=[]
cnt=0
error=0
for i,lyric in enumerate(result):
    try:
        if len(lyric)>500:
            sub=''
            spelled_sent = spell_checker.check(lyric[:500])
            spelled_sent=spelled_sent.as_dict()
            sub+=spelled_sent['checked']
            
            spelled_sent = spell_checker.check(lyric[500:])
            spelled_sent=spelled_sent.as_dict()
            sub+=spelled_sent['checked']
            
            lyrics.append(sub)

        else:
            spelled_sent=spell_checker.check(lyric)
            spelled_sent=spelled_sent.as_dict()
            lyrics.append(spelled_sent['checked'])
    except:
        lyrics.append(lyric)
        error+=1
    cnt+=1
    if cnt%100==0:
        print(cnt)

100
200
300
400
500
600
700


In [8]:
print(error)
print(len(lyrics))

0
746


normalizer 한번 더 적용

In [9]:
result=[]
for lyric in lyrics:
    result.append(repeat_normalize(lyric,num_repeats=1))
len(result)

746

In [10]:
data['Lyrics']=result
data

,Lyrics,label
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 나는 널 생각하...,0
1,라라랄라 라라랄라 이어 이어 네가 맛있었다 고 했던 살구빛 샐러드 맛있어서 또 먹고...,0
2,자꾸 마주치는 네 눈빛에 아기 아기 왠지 어색해 대체 왜 이래 고개를 숙여봐도 너의...,0
3,어깨동무까진 괜찮아 그 치마 손잡는 건 조금 위험해 잘 자란 메세 진 괜찮아그래도 ...,0
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 내 모습을 한번 ...,0
...,...,...
741,반쯤 미쳐가는 다 망가져 나의 삶넌 내게서 나를 지워가고 또 나를 부숴버려라 추락한...,3
742,무궁화 꽃이 피었습니다 우리 빼고 모두 다 멈춰지담 이 꽃이 피었습니다 현아 빼고 ...,3
743,아예 아예 아예 아예 아예 아예 아예 아예 아예 아예 왜 이렇게 감을 못 잡는 거야...,3
744,난 나쁜 계집애 나난 나쁜 계집애 난 나쁜 계집애 내 모든 나쁜 여자가 있는 고난 ...,3


In [11]:
data.to_csv('./data/Lyrics_check.csv',encoding='utf-8-sig',index=False)